In [190]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{LongType, StringType, StructField, StructType}
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{Column, DataFrame, Dataset, Row}
// import spark.implicits._

In [242]:
val record_schema = StructType(Array(
    StructField("ip", StringType, nullable=true),
    StructField("mac", StringType, nullable=true),
    StructField("hostname", StringType, nullable=true),
    StructField("serial_no", StringType, nullable=true),
    StructField("owner", StringType, nullable=true),
    StructField("source_name", StringType, nullable=true),
    StructField("record_id", LongType, nullable=false)
))

record_schema = StructType(StructField(ip,StringType,true), StructField(mac,StringType,true), StructField(hostname,StringType,true), StructField(serial_no,StringType,true), StructField(owner,StringType,true), StructField(source_name,StringType,true), StructField(record_id,LongType,false))


StructType(StructField(ip,StringType,true), StructField(mac,StringType,true), StructField(hostname,StringType,true), StructField(serial_no,StringType,true), StructField(owner,StringType,true), StructField(source_name,StringType,true), StructField(record_id,LongType,false))

In [255]:
val RecordId = "record_id"
def buildVerticesRDD(df: DataFrame): RDD[(VertexId, PartitionID)] = {
    def asVertexIdTuple(record: Row): (VertexId, PartitionID) = (record.getAs[VertexId](RecordId), 1)

    df.map(asVertexIdTuple).rdd
  }

RecordId = record_id


buildVerticesRDD: (df: org.apache.spark.sql.DataFrame)org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.PartitionID)]


record_id

In [243]:
val records = spark.read.format("csv")
                .option("header", "true")
                .schema(record_schema)
                .load("data/small_test_data.csv")
val mirrorColNames = for (col <- records.columns) yield "mirror_"+col.toString
// val mirrorColNames = mirror.columns.foreach(col => "mirror_"+col.toString)
val mirror = records.toDF(mirrorColNames: _*)

records = [ip: string, mac: string ... 5 more fields]
mirrorColNames = Array(mirror_ip, mirror_mac, mirror_hostname, mirror_serial_no, mirror_owner, mirror_source_name, mirror_record_id)
mirror = [mirror_ip: string, mirror_mac: string ... 5 more fields]


[mirror_ip: string, mirror_mac: string ... 5 more fields]

In [253]:
// records.printSchema
// records.show()
records.schema

StructType(StructField(ip,StringType,true), StructField(mac,StringType,true), StructField(hostname,StringType,true), StructField(serial_no,StringType,true), StructField(owner,StringType,true), StructField(source_name,StringType,true), StructField(record_id,LongType,true))

In [33]:
val edges = df1.join(df2,   df1("record_id").notEqual(df2("record_id")) && 
                            df1("ip").equalTo(df2("ip")) )
//                      df1("record_id") =!= df2("record_id") )
//                            && (df1("ip") <=> df2("ip"))) )
// select($"record_id")
edges.collect.foreach(println)

[106.60.189.138,null,bdruhan1m@1688.com,null,null,SY,4,106.60.189.138,4c:31:07:e9:9a:90,null,null,null,DT,5]
[106.60.189.138,4c:31:07:e9:9a:90,null,null,null,DT,5,106.60.189.138,null,bdruhan1m@1688.com,null,null,SY,4]


edges = [ip: string, mac: string ... 12 more fields]


[ip: string, mac: string ... 12 more fields]

In [105]:
val edges = records.join(mirror).where($"record_id".notEqual($"mirror_record_id") && 
                                       ($"ip".equalTo($"mirror_ip") ||
                                        $"mac".equalTo($"mirror_mac") ||
                                        $"hostname".equalTo($"mirror_hostname")
                                       ) 
                                      )
edges.select("record_id","mirror_record_id").distinct.show()
// .collect.foreach(println)

+---------+----------------+
|record_id|mirror_record_id|
+---------+----------------+
|        5|               4|
|        5|               3|
|        3|               5|
|        3|               4|
|        4|               5|
|        4|               3|
+---------+----------------+



edges = [ip: string, mac: string ... 12 more fields]


[ip: string, mac: string ... 12 more fields]

In [260]:
val edges = records.join(mirror, ($"record_id"=!=$"mirror_record_id" && 
                                  $"ip"===$"mirror_ip")
                        )
edges.select("record_id","mirror_record_id").distinct.show()

val asEdge: Row => Edge[PartitionID] = row => Edge[PartitionID](row.getAs[VertexId](0), row.getAs[VertexId](1), attr = 1)
val edgesRDD = edges.map(asEdge).rdd

+---------+----------------+
|record_id|mirror_record_id|
+---------+----------------+
|        4|               5|
|        5|               4|
+---------+----------------+



edges = [ip: string, mac: string ... 12 more fields]
asEdge = > org.apache.spark.graphx.Edge[org.apache.spark.graphx.PartitionID] = <function1>
edgesRDD = MapPartitionsRDD[558] at rdd at <console>:75


MapPartitionsRDD[558] at rdd at <console>:75

In [263]:
edgesRDD.collect()

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 188.0 failed 1 times, most recent failure: Lost task 0.0 in stage 188.0 (TID 1525, localhost, executor driver): java.lang.ClassCastException

Driver stacktrace:
StackTrace: Driver stacktrace:
  at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
  at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
  at org.apache.spark.scheduler.D

In [112]:
val edgeRDD = edges.select("record_id","mirror_record_id").withColumn("edge_id", monotonically_increasing_id).rdd

edgeRDD = MapPartitionsRDD[320] at rdd at <console>:49


MapPartitionsRDD[320] at rdd at <console>:49

In [ ]:
// val vertices: RDD[(VertexId, _)] =
// def asVertexIdTuple(record: Row): (VertexId, PartitionID) = (record.getAs[VertexId]("record_id"), 1)
records.withColumn("id", monotonically_increasing_id).select("id").map(r => r.getAs[Int](0)).rdd.collect()
// .rdd.collect
// .map(_.record_id)

In [254]:
// val nodes: RDD[(VertexId, (String, String))] = 
records
// .map( r => r(0) )
// .collect.foreach(println)

[ip: string, mac: string ... 5 more fields]

In [257]:
val nodes = buildVerticesRDD(records)

nodes = MapPartitionsRDD[540] at rdd at <console>:72


MapPartitionsRDD[540] at rdd at <console>:72

In [262]:
nodes.collect()

lastException: Throwable = null


Array((0,1), (1,1), (2,1), (3,1), (4,1), (5,1))

In [ ]:
val nodes: RDD[(VertexId, Option[String])] = records.select("record_id").flatMap(array => array).map((_.toLong, None))

In [114]:
val graph = Graph(vertices.rdd, edgeRDD)

Name: Unknown Error
Message: <console>:51: error: type mismatch;
 found   : org.apache.spark.rdd.RDD[org.apache.spark.sql.Row]
 required: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, ?)]
    (which expands to)  org.apache.spark.rdd.RDD[(Long, ?)]
Error occurred in an application involving default arguments.
       val graph = Graph(vertices.rdd, edgeRDD)
                                  ^
<console>:51: error: type mismatch;
 found   : org.apache.spark.rdd.RDD[org.apache.spark.sql.Row]
 required: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[?]]
Error occurred in an application involving default arguments.
       val graph = Graph(vertices.rdd, edgeRDD)
                                       ^

StackTrace: 

In [189]:
val users: RDD[(VertexId, (String))] =
  sc.parallelize(Array((3L, ("rxin")), (7L, ("jgonzal")),
                       (5L, ("franklin")), (2L, ("istoica"))))
// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Array(Edge(3L, 7L, "match"),    Edge(5L, 3L, "match"),
                       Edge(2L, 5L, "match"), Edge(5L, 7L, "match")))
// Define a default user in case there are relationship with missing user
// val defaultUser = ("John Doe", "Missing")
// Build the initial Graph
val graph = Graph(users, relationships)

users = ParallelCollectionRDD[450] at parallelize at <console>:61
relationships = ParallelCollectionRDD[451] at parallelize at <console>:65
graph = org.apache.spark.graphx.impl.GraphImpl@503343a8


org.apache.spark.graphx.impl.GraphImpl@503343a8